## Example: Learning the state transitions of a triple-dot array.

In this notebook we want to show how our algorithm can be used on a tripple-dot array to discover the transition of a given state. We choose this example, because it can be easily visualized in 3D still.


In [1]:
#import our simulation
from simulator import sim_NxM, Simulator

#for the algorithm
import numpy as np
from fit_zero_boundary import learn_zero_polytope
from fit_convex_polytope import learn_convex_polytope, generate_transitions_for_state, sample_boundary_points

#for plotting
import plotly.graph_objects as go
from scipy.spatial import HalfspaceIntersection
from scipy.optimize import linprog

#some helper code:

#function that finds a point inside a given polytope
def find_feasible_point(halfspaces):
    norm_vector = np.reshape(np.linalg.norm(halfspaces[:, :-1], axis=1), (halfspaces.shape[0], 1))
    c = np.zeros((halfspaces.shape[1],))
    c[-1] = -1
    A = np.hstack((halfspaces[:, :-1], norm_vector))
    b = - halfspaces[:, -1:]
    res = linprog(c, A_ub=A, b_ub=b, bounds = (None,None))
    return res.x[:-1]

def plot_facet(halfspaces, labels = None, points = None, text_dist_multiplier=1.0):
    startpoint = find_feasible_point(halfspaces)
    vertices=HalfspaceIntersection(halfspaces, startpoint).intersections
    
    
    
    fig = go.Figure()
    fig.add_trace(go.Mesh3d(x=vertices[:, 2],
                           y=vertices[:, 1],
                           z=vertices[:, 0],
                           color="yellow",
                           opacity=1.0,
                           alphahull=0))
    for i in range(halfspaces.shape[0]):
        Ai = halfspaces[i,:-1]
        bi = halfspaces[i,-1]
        distance = np.max(np.abs(vertices@Ai.reshape(-1,1)+ bi.reshape(1,-1)),axis=1)
        vi = vertices[distance < 1.e-5]

        mean = np.mean(vi,axis=0)
        
        #add annotation
        if not labels is None:
            text_pos = mean + text_dist_multiplier*0.07*Ai/np.linalg.norm(Ai)
            arrow_end = mean + text_dist_multiplier*0.05*Ai/np.linalg.norm(Ai)
            text = str(labels[i])
            #annotations.append(annotation)
            fig.add_trace(go.Scatter3d(x=mean[2:3],
                            y=mean[1:2],
                            z=mean[0:1],
                            showlegend=False,
                            mode="markers",
                            marker=dict(color= 'black',size=3)))
            fig.add_trace(go.Scatter3d(x=[mean[2],arrow_end[2]],
                        y=[mean[1],arrow_end[1]],
                        z=[mean[0],arrow_end[0]],
                        mode='lines',
                        showlegend=False,
                        line=dict(color= 'black', width=2)))
            fig.add_trace(go.Scatter3d(x=text_pos[2:3],
                            y=text_pos[1:2],
                            z=text_pos[0:1],
                            showlegend=False,
                            textfont=dict(
                                color="black",
                                size=14
                            ),
                            mode="text", textposition="middle center",
                            marker=dict(color= 'black',size=3), text=[text]))
        #plot mesh
        for j in range(vi.shape[0]):
            for k in range(j+1,vi.shape[0]):
                #check if midpoint is inside to filter out lines inside a facet.
                mid = (vi[j]+vi[k])/2
                di = halfspaces[:,:-1]@mid+ halfspaces[:,-1]
                di[i] = -1
                if np.all(di<-1.e-5):
                    continue
                fig.add_trace(go.Scatter3d(x=vi[[j,k], 2],
                        y=vi[[j,k], 1],
                        z=vi[[j,k], 0],
                        mode='lines',
                        showlegend=False,
                        line=dict(color= 'black', width=2)))
    
    if not points is None:
        fig.add_trace(go.Scatter3d(x=points[:,2],
                            y=points[:,1],
                            z=points[:,0],
                            showlegend=False,
                            mode='markers', marker=dict(color= 'red',size=3)))
    fig.update_layout(scene=dict( xaxis_title=r'V<sub>3</sub>', yaxis_title=r'V<sub>2</sub>',zaxis_title=r'V<sub>1</sub>'))
    fig.show()

## Step 1: initializing the simulation

First, we create the simulation. We simulate a 2x2 array according to the constant interaction model in the (1 0 0 1) state

In [8]:

#set random state for reproducibility
np.random.seed(0)

delta = 1.0/1000 #precision of the line-search. 1mV
rho=3.0/10 #setting of rho in the paper as a measure of simulation complexity. this is what rho=3 in the paper is
sim = sim_NxM(2, 2, delta, rho) #simulation of an 2x2 array.

#compute Gamma. For simplicity, we don't check the rodering. it will most likely be correct.
sim.activate_state(np.zeros(4))
sim.set_reservoir(True)
num_start_samples = 4*sim.num_dots*(sim.num_dots+5) #number of initial line-searches (same as in the paper)
halfspaces, _,_,_ = learn_zero_polytope(sim, delta, num_start_samples = num_start_samples)
halfspaces /= np.linalg.norm(halfspaces[:,:-1],axis=1).reshape(-1,1) #normalize to unit length normals
gamma = halfspaces[:,:-1]

## Computing Gamma

To compute gamma, we first learn the boundaries of the state (0,0,0,0). We then label the learned transitions based on which we think adds one electron to the i-th dot.

For a quick verification that the learned boundaries are correct, we compute the relative error of each element. 

In [9]:
#now disable the reservoir, we don't need it anymore
sim.set_reservoir(False)
#initialize device in target state
target_state=np.array([1,0, 0, 1],dtype=np.int64)
sim.activate_state(target_state)

#now compute the projection of the simulation to remove the axis where the polytopes in the reservoir-free
#array are unbounded. this allows us to plot the 3d projection of the 4d facet
def compute_projection(sim, gamma):
    # Compute v such, that for any t with sum(t)=0: t^TAv = 0
    v = np.linalg.inv(gamma)@np.ones(sim.num_dots)
    v /= np.linalg.norm(v)
    
    # Project away the vector direction v
    # We remove the vector by defining a householder transformation that would move v->(1,0,...,0)
    # and then we remove the first element (we make use of the fact that the householder transformation is its own inverse)
    h = v.copy()
    h[0] -= 1.0
    h /= np.linalg.norm(h)
    P = np.eye(sim.num_dots) - 2*np.outer(h,h)
    P = P[:,1:]

    sim_proj = sim.slice(P, np.zeros(sim.num_dots))
    gamma_proj = gamma@P
    return sim_proj, gamma_proj

sim_proj_est, gamma_proj = compute_projection(sim,gamma)
sim_proj_real, _ = compute_projection(sim, sim.Cinv@sim.C_g)

#visualize both projections

#first the true one
poly_real = sim_proj_real.boundaries()
print("projected facet labels")
print(poly_real.labels)
print("Real facet after projection")
halfspaces=np.hstack([poly_real.A,poly_real.b.reshape(-1,1)])
#polytope not bounded, so add lower bounds as well
lower_bound = np.hstack([sim_proj_real.boundsA,sim_proj_real.boundsb.reshape(-1,1)])
halfspaces = np.vstack([halfspaces,lower_bound])
labels = np.vstack([poly_real.labels,np.zeros((4,4),dtype=np.int64)])
plot_facet(halfspaces, labels, text_dist_multiplier=5.0)

print("Facet after projection using estimated Gamma")
#now using the estimated gamma
poly_est = sim_proj_est.boundaries()
halfspaces=np.hstack([poly_est.A,poly_est.b.reshape(-1,1)])
#polytope not bounded, so add lower bounds as well
lower_bound = np.hstack([sim_proj_est.boundsA,sim_proj_est.boundsb.reshape(-1,1)])
halfspaces = np.vstack([halfspaces,lower_bound])
labels = np.vstack([poly_est.labels,np.zeros((4,4),dtype=np.int64)])
plot_facet(halfspaces, labels, text_dist_multiplier=5.0)

projected facet labels
[[-1.  1.  1. -1.]
 [-1.  0.  0.  1.]
 [-1.  0.  1.  0.]
 [-1.  1.  0.  0.]
 [ 0.  0.  1. -1.]
 [ 0.  1.  0. -1.]
 [ 1.  0.  0. -1.]]
Real facet after projection


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:182: RuntimeWarning:

invalid value encountered in true_divide



Facet after projection using estimated Gamma


### Now learn all of the other polytopes that are relevant to the target states

We first search for all transitions in the state. for this, we call ``generate_transitions_for_state`` with arguments ``max_k=3`` and ``max_moves = 2``. ``max_k`` indicates the maximum number of sites in the array that can be changed by a transition. ``max_moves`` limits the amount of electrons that can move in a transition. e.g., transition (1,-1,1) has at least 2 moving electrons: one moving from the middle to the sides and then another added to the array.

Afterwards we plot the learned facet and repeat the array with only one electron moving at most (And thus at most 2 sites can be affected). This excludes two facets in our example and we show that we can still learn the polytope reasonably well. In this example we also plot the points our algorithm took during its run

In [10]:
# Set a seed for reproducability
np.random.seed(0)

#we now sue the projected simulation to learn its facets

# Generate the transitions we want to look for
T = generate_transitions_for_state(target_state, max_k=4, max_moves = 2,has_reservoir=False)
print("searching for transitions")
print(T)

# we need a starting point to create a polytope
#this is slightly cheating, since we compute a point from the ground turth polytope
startpoint = sim_proj_est.boundaries().point_inside
#this is less cheating and how we do it in the paper.
#startpoint, _, _ = sim.line_search(sim.boundaries().point_inside, np.random.randn(sim.num_inputs))
#startpoint = 0.95*startpoint + 0.05 * sim.boundaries().point_inside
print("start point:", startpoint)
# Learn the polytope (set verbose = 2 for more output, such as the convergence of the max likelihood)
# note that we use here the projected simulation and the projected gamme.
A_res, b_res, x_m, x_p, found, num_searches = learn_convex_polytope(sim_proj_est, delta, startpoint, T.astype(float), gamma_proj, max_searches=500, verbose=1)

searching for transitions
[[-1  0  0  1]
 [-1  0  1  0]
 [-1  1  0  0]
 [ 0  0  1 -1]
 [ 0  1  0 -1]
 [ 1  0  0 -1]
 [-1  1  1 -1]]
start point: [-0.06644321 -0.05158848  1.53701152]
Number of searches:  37 / 500
Number of searches:  56 / 500
Number of searches:  71 / 500
Number of searches:  86 / 500
Number of searches:  98 / 500
Number of searches:  110 / 500
Finished learning polytope
Number of transitions found: 7
max_rad not found: 0.0
[-1.  0.  0.  1.] 17 1.0063778017688698
[-1.  0.  1.  0.] 15 0.06717569063654799
[-1.  1.  0.  0.] 17 0.0672543687357991
[ 0.  0.  1. -1.] 18 0.06739480399456954
[ 0.  1.  0. -1.] 18 0.06758196575792164
[ 1.  0.  0. -1.] 17 1.0040832203535517
[-1.  1.  1. -1.] 9 0.010593288253030617


In [11]:
print("Plot of learned projected facet")
#now using the estimated gamma
halfspaces=np.hstack([A_res[found],b_res[found].reshape(-1,1)])
#polytope not bounded, so add lower bounds as well
lower_bound = np.hstack([sim_proj_est.boundsA,sim_proj_est.boundsb.reshape(-1,1)])
halfspaces = np.vstack([halfspaces,lower_bound])
labels = np.vstack([T[found],np.zeros((4,4),dtype=np.int64)])
plot_facet(halfspaces, labels,points=x_p, text_dist_multiplier=5.0)

Plot of learned projected facet
